In [190]:
import numpy as np
from glob import glob
import json
import networkx as nx
from networkx.readwrite import json_graph
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import preprocessing
from karateclub import Graph2Vec
from sklearn.linear_model import SGDClassifier
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from itertools import cycle, islice
import creme
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier, SGDOneClassSVM
from collections import defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from scipy.special import softmax
np.set_printoptions(precision=3)

pd.set_option('display.max_colwidth', None)

std = StandardScaler()


In [206]:
with open('data/REDDIT-BINARY_graph2vec_classification_train.npy', 'rb') as f:
    redit_train = np.load(f, allow_pickle=True)

with open('data/REDDIT-BINARY_graph2vec_classification_test.npy', 'rb') as f:
    reddit_test = np.load(f, allow_pickle=True)

with open('data/PROTEINS_graph2vec_classification_test.npy', 'rb') as f:
    unknown_test = np.load(f, allow_pickle=True)

with open('data/PROTEINS_graph2vec_classification_train.npy', 'rb') as f:
    unknown_train = np.load(f, allow_pickle=True)


with open('data/IMDB-BINARY_graph2vec_classification_test.npy', 'rb') as f:
    unknown_test_imdb = np.load(f, allow_pickle=True)

with open('data/IMDB-BINARY_graph2vec_classification_train.npy', 'rb') as f:
    unknown_train_imdb = np.load(f, allow_pickle=True)


# SGDclassifier
- When inclue multiclass, automaticaly use OvA
- Able to get score percentage by using min-max normalisation

# Reference
- https://www.youtube.com/watch?v=5KyH6v8oKNQ

In [230]:
def train_classif(model, epoch, x_embed, y_label, unique_lbl):
    for i in range(epoch):
        model.partial_fit(x_embed, y_label, unique_lbl)

    return model

def predict_classif(model, x_test_embed):
    y_pred = model.predict([x_test_embed]).squeeze()
    y_proba = model.decision_function([x_test_embed])
    y_proba = (y_proba - y_proba.min()) / (y_proba.max() - y_proba.min())
    y_proba = [x / np.sum(y_proba) for x in y_proba.squeeze()]
    y_proba = y_proba[y_pred] * 100
    y_proba = np.round(y_proba, 3)
    return y_pred, y_proba


In [231]:
# Train with REDDIT data as unauth access+data theft with PROTEINS(train and test so have more sample->more accurate) as unknown
unknown_protein_label = []
# Replace with synthetic label for PROTEINS to equate it to unknwon type class(2==unknwon)
for  y in [y for x, y, g in unknown_test] + [y for x, y, g in unknown_train]:
    if y == 0:
        unknown_protein_label.append(2)
    if y == 1:
        unknown_protein_label.append(2)

X_train = [x for x, y, g in redit_train] + [x for x, y,
                                            g in unknown_test] + [x for x, y, g in unknown_train]
std_model = std.fit(X_train)
X_train = std_model.transform(X_train)
y_train = [y for x, y, g in redit_train] + unknown_protein_label

X_test = [x for x, y, g in unknown_test]
X_test = std_model.transform(X_test)
y_test = unknown_protein_label


In [232]:
# Test to check if given unknwon data, it should predict only 2, never 0 or 1
model = SGDClassifier(random_state=47, tol=1e-3)
model = train_classif(model, 100, X_train, y_train, [0, 1, 2])
pred, score = predict_classif(model, X_test[2])
print("predicted:",pred, " with score:",score, " actual:", y_test[2])


predicted: 2  with score: 74.006  actual: 2


In [220]:
# Test to check if given knwon data, it should predict 0 or 1, never 2
model = SGDClassifier(random_state=47, tol=1e-3)
model = train_classif(model, 100, X_train, y_train, [0, 1, 2])
pred, score = predict_classif(model, X_train[2])
print("predicted:", pred, " with score:", score, " actual:", y_train[2])


[1.000e+00 3.646e-85 1.485e-51]
[0.7152863107608453, 0.0, 0.2847136892391548]
predicted: 0  with score: 71.53  actual: 0


In [221]:
# Now train with another data IMDB(train only) to test if the data is predicted as unknown
unknown_train_label_imdb = []
for x,y,g in unknown_train_imdb:
    if y == 0:
        unknown_train_label_imdb.append(2)
    if y == 1:
        unknown_train_label_imdb.append(2)

X_train_withunknown = [x for x, y, g in unknown_train_imdb]
X_train_withunknown = std_model.transform(X_train_withunknown)
y_train_withunknown = unknown_train_label_imdb


In [233]:
# Test to check if given unknwon data, it should predict only 2, never 0 or 1
model = SGDClassifier(random_state=47, tol=1e-3)
model = train_classif(model, 100, X_train, y_train, [0, 1, 2])
pred, score = predict_classif(model, X_train_withunknown[0])
print("predicted:", pred, " with score:",
      score, " actual:", y_train_withunknown[0])


predicted: 2  with score: 64.424  actual: 2


In [224]:
# Finally test with known sample REDDIT(test) to check if predicts either of the known classes 0 or 1
X_train_known = [x for x, y, g in reddit_test]
X_train_known = std_model.transform(X_train_known)
y_train_known = [ y for x, y, g in reddit_test]


In [229]:
# Test to check if given knwon data, it should predict 0 or 1, never 2
model = SGDClassifier(random_state=47, tol=1e-3)
model = train_classif(model, 100, X_train, y_train, [0, 1, 2])
pred, score = predict_classif(model, X_train_known[4])
print("predicted:", pred, " with score:",
      score, " actual:", y_train_known[4])


[1.000e+000 0.000e+000 1.697e-201]
[0.5838577380086818, 0.0, 0.41614226199131826]
predicted: 0  with score: 58.39  actual: 0


# Using dataset from custodio gen data

In [46]:
folder = glob('slade_data\\*')
class1 = []
class2 = []

for file in folder:
    casetype = file.split('\\')[1].split('-')[0]
    if 'ipnet' in casetype:
        graphjson = json.load(open(file,'r'))
        G = json_graph.node_link_graph(graphjson)
        H = nx.convert_node_labels_to_integers(G)
        class1.append(H)
    if 'smtp' in casetype:
        graphjson = json.load(open(file,'r'))
        G = json_graph.node_link_graph(graphjson)
        H = nx.convert_node_labels_to_integers(G)
        class2.append(H)

In [89]:
packclass = class1 + class2
model = Graph2Vec(dimensions=64, epochs=50)
model.fit(packclass)
packclassembelist = model.get_embedding()
packclassembelist = scaler.fit_transform(packclassembelist)

train_X = np.array([packclassembelist[0], packclassembelist[1], packclassembelist[3] , packclassembelist[4]])
train_y = [0,0,1,1]

test_X = np.array([packclassembelist[2] , packclassembelist[5] , packclassembelist[6]])
test_y = [0,1,1]

In [90]:
from sklearn.cluster import SpectralClustering
clustering = SpectralClustering(n_clusters=2,
        assign_labels='discretize',
        random_state=0).fit(packclassembelist)
clustering.labels_

array([1, 1, 0, 0, 0, 1, 0], dtype=int64)

In [91]:
model = SGDClassifier(random_state=47, tol=1e-3)
model = train_classif(model, 100, train_X, train_y, [0, 1])

In [92]:
pred, score = predict_classif(model, test_X[0])
print(test_y[0])
print(pred, score)

0
1 [16.7241284]


In [93]:
print(model.score(test_X,test_y))

0.6666666666666666


# Experiment dump

In [100]:
def get_similairty_graph(G, H):
    """
    to get D3 graph data object for rendering to see graph similarity
    """
    GH = nx.compose(G, H)
    # set edge colors
    edge_colors = dict()
    for u, v, attr in GH.edges(data=True):
        if G.has_edge(u, v):
            if H.has_edge(u, v):
                GH[u][v]['color'] = 'magenta'
                GH[u][v]['size'] = 5
                continue
            #GH[u][v]['color'] = 'lightgreen'
        #elif H.has_edge(u, v):
            #GH[u][v]['color'] = 'lightblue'

    # # set node colors
    # G_nodes = set(G.nodes())
    # H_nodes = set(H.nodes())
    # node_colors = []
    # for node in GH.nodes():
    #     if node in G_nodes:
    #         if node in H_nodes:
    #             node_colors.append('magenta')
    #             continue
    #         node_colors.append('lightgreen')
    #     if node in H_nodes:
    #         node_colors.append('lightblue')

    graph2json = json_graph.node_link_data(GH)
    graph2json.pop('directed', None)
    graph2json.pop('multigraph', None)
    graph2json.pop('graph', None)

    return graph2json

In [101]:
grasdasd = get_similairty_graph(class1[0], class2[0])

In [102]:
json.dump(grasdasd,open('asdasd.json', 'w'))